In [1]:
# Setup
from conf_tools import GlobalConfig
GlobalConfig.global_load_dir("..")
# languages
from mocdp.lang import parse


INFO:conf_tools:loaded global config dir '..'


This is an example of a model:

In [2]:
a = parse("""
cdp {
        provides f [R]
        requires z [R]
        
        hnlin = dp {
            provides x [R]
            requires r [R]
            
            implemented-by load SimpleNonlinearity1
        }
        
        hnlin.x >= max(f, hnlin.r)
        z >= hnlin.r 
        
    }
""")

In [3]:
print a.desc()

SimpleWrap
 provides          f (R) 
 requires          z (R) 
 requires        max (R) 
| DPLoop0:   R -> R×R
| L Series:   R×(R×R) -> R×R
| L S1 Mux(R×(R×R) → ((R×))×((R×)), [[(1, 1)], [0]])
| L S2 Series:   ((R×))×((R×)) -> R×R
| L S2 S1 Series:   ((R×))×((R×)) -> ((R×))×((R×))
| L S2 S1 S1 Series:   ((R×))×((R×)) -> ((R×))×(R×R)
| L S2 S1 S1 S1 Parallel:   ((R×))×((R×)) -> (R×R)×((R×))
| L S2 S1 S1 S1 P1 Series:   (R×) -> R×R
| L S2 S1 S1 S1 P1 S1 Mux((R×) → R, 0)
| L S2 S1 S1 S1 P1 S2 Series:   R -> R×R
| L S2 S1 S1 S1 P1 S2 S1 SimpleNonlinearity1(R→R)
| L S2 S1 S1 S1 P1 S2 S2 Mux(R → R×R, [(), ()])
| L S2 S1 S1 S1 P2 Id((R×))
| L S2 S1 S1 S2 Mux((R×R)×((R×)) → ((R×))×(R×R), [[(0, 1)], [(1, 0), (0, 0)]])
| L S2 S1 S2 Parallel:   ((R×))×(R×R) -> ((R×))×((R×))
| L S2 S1 S2 P1 Id((R×))
| L S2 S1 S2 P2 Series:   R×R -> (R×)
| L S2 S1 S2 P2 S1 Max(R)
| L S2 S1 S2 P2 S2 Mux(R → (R×), [()])
| L S2 S2 Mux(((R×))×((R×)) → R×R, [(0, 0), (1, 0)])


In [4]:
b = parse("""
    cdp {
        provides mission_time  [s]
        provides extra_payload [g]
        
        battery = dp {
            provides capacity [J]
            requires weight   [g]
            
            implemented-by load BatteryDP
        }
        
        actuation = dp {
            provides payload [g]
            requires power   [W]
            
            implemented-by code mocdp.example_battery.Mobility
        }
                
        capacity provided by battery >= mission_time * (power required by actuation)    
        payload provided by actuation >= (weight of battery) + extra_payload
    }
""")

In [5]:
print b.desc()

SimpleWrap
 provides mission_time (R[s]) 
 provides extra_payload (R[g]) 
 requires     weight (R[g]) 
| DPLoop0:   R[s]×R[g] -> R[g]
| L Series:   (R[s]×R[g])×R[g] -> R[g]
| L S1 Mux((R[s]×R[g])×R[g] → (R[g]×R[g])×((R[s]×)), [[(0, 1), 1], [(0, 0)]])
| L S2 Series:   (R[g]×R[g])×((R[s]×)) -> R[g]
| L S2 S1 Series:   (R[g]×R[g])×((R[s]×)) -> R[J]
| L S2 S1 S1 Series:   (R[g]×R[g])×((R[s]×)) -> R[s]×R[W]
| L S2 S1 S1 S1 Parallel:   (R[g]×R[g])×((R[s]×)) -> ((R[W]×))×((R[s]×))
| L S2 S1 S1 S1 P1 Series:   R[g]×R[g] -> (R[W]×)
| L S2 S1 S1 S1 P1 S1 Series:   R[g]×R[g] -> R[W]
| L S2 S1 S1 S1 P1 S1 S1 Series:   R[g]×R[g] -> R[g]
| L S2 S1 S1 S1 P1 S1 S1 S1 Mux(R[g]×R[g] → R[g]×R[g], [1, 0])
| L S2 S1 S1 S1 P1 S1 S1 S2 Sum(R[g])
| L S2 S1 S1 S1 P1 S1 S2 Mobility(R[g]→R[W])
| L S2 S1 S1 S1 P1 S2 Mux(R[W] → (R[W]×), [()])
| L S2 S1 S1 S1 P2 Id((R[s]×))
| L S2 S1 S1 S2 Mux(((R[W]×))×((R[s]×)) → R[s]×R[W], [(1, 0), (0, 0)])
| L S2 S1 S2 Multiply(R[s]×R[W]→R[J])
| L S2 S2 BatteryDP(R[J]→R[g])


In [7]:
b = parse("""
    cdp {
        provides x  [s]
        provides y  [s]
        requires z  [s]
                
        z >= max(x, y)
    }
""")

In [9]:
print b.desc()

SimpleWrap
 provides          y (R[s]) 
 provides          x (R[s]) 
 requires          z (R[s]) 
| Series:   R[s]×R[s] -> R[s]
| S1 Mux(R[s]×R[s] → R[s]×R[s], [1, 0])
| S2 Max(R[s])
